In [ ]:
import pandas as pd

In [ ]:
schedule_path = "/mnt/idms/fberes/network/roland_garros/data/schedule_df.csv"
schedule_df = pd.read_csv(schedule_path, sep="|")
schedule_df.head(3)

In [ ]:
names = set(schedule_df["playerName active"]).union(set(schedule_df["playerName opponent"]))
names = set([n.lower() for n in names])

In [ ]:
names_parts = []
for n in names:
    names_parts.append(n.replace(" ",""))
    names_parts += n.split(" ")
len(names_parts)

In [ ]:
WORDS = []
word_file_path = "/mnt/idms/fberes/network/roland_garros/data/rg17_8000_important_en_words_plus_players.txt"
with open(word_file_path) as wf:
    for line in wf:
        w = line.rstrip()
        if "@" not in w and w not in names_parts:
            WORDS.append(w)
WORDS = set(WORDS)

In [ ]:
len(WORDS)

In [ ]:
tweet_file_path = "/mnt/idms/fberes/network/roland_garros/data/rg17_tweets_eng.csv"
tweets_df = pd.read_csv(tweet_file_path, sep="|")

In [ ]:
tweets_df.head()

In [ ]:
import re
def clean_text(t):
    clean_1 = ' '.join(re.findall("[\w,\@]+",t))
    clean_2 = ' '.join(re.findall("[^\,,\d]+",clean_1))
    return clean_2.lower()

In [ ]:
tweets_df["text_clean"] = tweets_df["text"].apply(clean_text)

In [ ]:
tweets_df["text_clean_splitted"] = tweets_df["text_clean"].apply(str.split)

In [ ]:
tweets_df["filtered_words"] = tweets_df["text_clean_splitted"].apply(lambda x: list(WORDS.intersection(set(x))))

In [ ]:
tweets_df = tweets_df[tweets_df["filtered_words"].apply(lambda x: len(x)>1)]

In [ ]:
len(tweets_df)

In [ ]:
filtered_tweet_texts = list(tweets_df["filtered_words"])
filtered_tweet_texts[:2]

# TEST

In [ ]:
import gensim, logging

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
model = gensim.models.Word2Vec(filtered_tweet_texts, min_count=5, batch_words=100)

In [ ]:
model.most_similar(positive=["win"],topn=20)

In [ ]:
model.most_similar(positive=["beat"],topn=20)

In [ ]:
model.most_similar(positive=["set"],topn=20)

model.wv.vocab